In [ ]:
#For the psychopy csv:
#"Stimulus" is the name of the csv column for event type
#"Marker Timestamp" is the name of the csv column for when the event occured
#These are present in 2 places in the code.
#The names should be changed in the code or in the csv if different names are being used
#The following locations are the bits of code that mention these columns:

"""
#First in the load_behav function:

beh = pd.read_csv(behav_csv)
beh = beh.dropna(subset=['Marker Timestamp', 'Stimulus'])                #This line!!

beh['marker_ts'] = pd.to_datetime(
    beh['Marker Timestamp'], utc=True                                    #This line!!
).apply(lambda ts: ts.timestamp())

"""

"""
#Second when generating the marker map

unique_stimuli = beh_df['Stimulus'].unique()                             #This line!!
marker_map = {stim: i + 1 for i, stim in enumerate(unique_stimuli)}

"""

#Change those lines further down in the actual code as needed!

In [ ]:
!pip install mne PyWavelets


In [ ]:
import numpy as np
import pandas as pd
import pywt
import mne
from scipy import signal, integrate
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
import os
import matplotlib.pyplot as plt


In [ ]:
#Make sure only forward slashes are used!! 
#Also, be careful to copy the file name exactly! Any extra characters, even a space at the end, and it will give you an error message!

eeg_file = r"/Users/username/Desktop/folder/eeg_data.csv" #replace with the filepath to your eeg csv
behav_file = r"/Users/username/Desktop/folder/psychopy_data.csv" #replace with the filepath to your psychopy csv

# ✅ Load the files
eeg_df = load_eeg(eeg_file)
beh_df = load_behav(behav_file)


In [ ]:
beh_df_raw = pd.read_csv(behav_file)
behav_df_raw = pd.read_csv(behav_file)


In [ ]:
def load_eeg(eeg_csv: str) -> pd.DataFrame:
    df = pd.read_csv(eeg_csv, sep=None, engine='python')
    df['timestamps'] = df['timestamps'].astype(float)
    return df

def load_behav(behav_csv: str, ts_col: str = 'Marker Timestamp') -> pd.DataFrame:
    """
    Reads the psychopy behavioral CSV, converts timestamps to Unix seconds.
    """
    beh = pd.read_csv(behav_csv)
    beh['marker_ts'] = (
        pd.to_datetime(beh[ts_col])
          .astype(np.int64) / 1e9
    )
    return beh.dropna(subset=['marker_ts'])


beh_df = behav_df_raw.copy()
beh_df["marker_ts"] = beh_df["Marker Timestamp"]  # Already in Unix seconds, so just rename




print("EEG timestamps range:")
print("Start:", eeg_df["timestamps"].min())
print("End  :", eeg_df["timestamps"].max())

print("\nBehavioral timestamps (marker_ts) range:")
print("Start:", beh_df["marker_ts"].min())
print("End  :", beh_df["marker_ts"].max())

# Calculate intersection
start_ts = max(eeg_df["timestamps"].min(), beh_df["marker_ts"].min())
end_ts   = min(eeg_df["timestamps"].max(), beh_df["marker_ts"].max())
print("\nCommon overlap window:")
print("Start:", start_ts)
print("End  :", end_ts)

# Extract EEG slice that matches
eeg_sync = eeg_df[(eeg_df["timestamps"] >= start_ts) & (eeg_df["timestamps"] <= end_ts)]
print(f"\n🧪 EEG samples in overlap window: {len(eeg_sync)}")


def sync_and_mark(
    eeg_df: pd.DataFrame,
    beh_df: pd.DataFrame,
    marker_map: dict,
    fs: int = 256
) -> pd.DataFrame:
    start_ts = max(eeg_df['timestamps'].min(), beh_df['marker_ts'].min())
    end_ts   = min(eeg_df['timestamps'].max(), beh_df['marker_ts'].max())
    eeg_sync = eeg_df.query("@start_ts <= timestamps <= @end_ts").copy()
    eeg_sync['marker'] = 0
    times = eeg_sync['timestamps'].values
    for _, row in beh_df.iterrows():
        code = marker_map.get(row['Marker'], 0)
        idx  = np.argmin(np.abs(times - row['marker_ts']))
        eeg_sync.at[eeg_sync.index[idx], 'marker'] = code
    return eeg_sync.reset_index(drop=True)


In [ ]:

from datetime import datetime

# Rename for consistency with extended pipeline
beh_df_raw["marker_ts_raw"] = beh_df_raw["Marker Timestamp"]

# Apply simple Unix-to-datetime conversion (no offset)
beh_df_raw["marker_ts"] = pd.to_datetime(beh_df_raw["marker_ts_raw"], unit='s')

beh_df = beh_df_raw.copy()
beh_df["marker_ts"] = beh_df["marker_ts_raw"]

In [ ]:
def apply_ica(raw, n_components, eog_thresh=2.5):
    # Skip ICA if data is too short or variance is degenerate
    data = raw.get_data()
    if data.shape[1] < 20 or np.isnan(data).any() or np.allclose(data, 0):
        print("⚠️ Skipping ICA: insufficient data duration or invalid signal")
        return raw

    # Proceed with ICA
    ica = mne.preprocessing.ICA(
        n_components=n_components,
        max_iter='auto',
        random_state=0
    )
    try:
        ica.fit(raw)
    except Exception as e:
        print(f"⚠️ ICA failed: {e}")
        return raw

    # Optionally exclude EOG-like components (if present)
    try:
        eog_indices, scores = ica.find_bads_eog(raw, threshold=eog_thresh)
        ica.exclude = eog_indices
        raw = ica.apply(raw.copy())
    except Exception as e:
        print(f"⚠️ EOG removal skipped: {e}")

    return raw



def wavelet_denoise(sig, wavelet='db4', level=4):
    coeffs = pywt.wavedec(sig, wavelet, level=level)
    sigma  = np.median(np.abs(coeffs[-level])) / 0.6745
    uthresh= sigma * np.sqrt(2*np.log(len(sig)))
    coeffs[1:] = [pywt.threshold(c, uthresh, mode='hard') for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)[:len(sig)]


In [ ]:
class Recording:
    def __init__(
        self,
        eeg_df: pd.DataFrame,
        beh_df: pd.DataFrame,
        marker_map: dict,
        length: float = 20,
        bands: dict = None,
        fs: int = 256,
        min_freq: float = 1,
        max_freq: float = 30
    ):
        self.fs = fs
        self.length = length
        self.marker_map = marker_map
        # reverse map: code → label
        self.marker_dict = {v:k for k,v in marker_map.items()}
        self.bands = bands or {
            "delta": [0.5,4], "theta":[4,8],
            "alpha":[8,12],"beta":[12,30]
        }
        self.beh = beh_df  # store behavioral data for diagnostics


        # 1. Sync streams & inject markers
        merged = sync_and_mark(eeg_df, beh_df, marker_map, fs)
        picks = ['TP9','AF7','AF8','TP10']
        
        # 2. Create MNE Raw, filter, ICA
        data = merged[picks].T.values * 1e-6  # µV → V
        info = mne.create_info(picks, fs, ['eeg']*4)
        raw  = mne.io.RawArray(data, info, verbose=False)
        raw.filter(min_freq, max_freq, method='iir', verbose=False)
        # use exactly as many ICA components as you have channels
        
        
        n_ica_components = min(len(picks), raw.info['nchan'], raw.get_data().shape[0])
        raw = apply_ica(raw, n_components=n_ica_components)


        # 3. Update dataframe with cleaned data (µV)
        clean = (raw.get_data().T * 1e6)
        for i,ch in enumerate(picks):
            merged[ch] = clean[:,i]

        self.df = merged
        self._epoch_fixed_length(picks)
        self._clean_windows(threshold=200) # CHANGE THE THRESHOLD NUMBER DEPENDING ON HOW EXACT YOU WANT THE RESULTS
        self._compute_welch()
        self._compute_band_power()

    def _epoch_fixed_length(self, picks):
        n_pts = int(self.length * self.fs)
        self.events = {code:{ch:[] for ch in picks} for code in self.marker_dict}
        idxs = self.df.index[self.df['marker']!=0]
        for idx in idxs:
            code = int(self.df.at[idx,'marker'])
            for ch in picks:
                segment = self.df[ch].iloc[idx: idx+n_pts].values
                if len(segment)==n_pts:
                    self.events[code][ch].append(segment)

    def _clean_windows(self, threshold=100):
        self.windows = {c:{ch:[] for ch in self.events[c]} for c in self.events}
        size = n = int(self.length*self.fs//4)
        for c in self.events:
            for ch, segs in self.events[c].items():
                for seg in segs:
                    # split into 4 sub-windows
                    for j in range(0, len(seg), size):
                        w = seg[j:j+size]
                        if np.max(np.abs(w)) < threshold:
                            self.windows[c][ch].append(w)

    def _compute_welch(self):
        self.freqs = None
        self.psd = {c: {ch: None for ch in self.windows[c]} for c in self.windows}
        win = int(self.length * self.fs)
    
        for c in self.windows:
            for ch, ws in self.windows[c].items():
                psds = []
                freqs_list = []
    
                for w in ws:
                    f, Pxx = signal.welch(w, fs=self.fs, nperseg=win)
                    psds.append(Pxx)
                    freqs_list.append(np.round(f, 3))  # Round to make comparable
    
                if not psds:
                    continue  # Skip if no valid windows
    
                # Use the most common frequency array
                freq_keys = [tuple(f) for f in freqs_list]
                most_common_freq_key = pd.Series(freq_keys).value_counts().idxmax()
                most_common_freq = np.array(most_common_freq_key)
    
                self.freqs = most_common_freq
                matched_psds = [ps for ps, f in zip(psds, freqs_list) if np.allclose(f, most_common_freq)]
    
                self.psd[c][ch] = np.mean(matched_psds, axis=0) if matched_psds else None
    
                    

    def _compute_band_power(self):

        if self.freqs is None:
            print("⚠️ No frequency data available — skipping band power computation.")
            self.relative_power = {}
            self.band_df = pd.DataFrame()
            return
        
        res = {}
        df = {}
        freq_res = self.freqs[1]-self.freqs[0]
        for c in self.psd:
            res[c] = {}

            for ch, P in self.psd[c].items():
                if not isinstance(P, np.ndarray) or P.size == 0:
                    print(f"⚠️ Skipping '{ch}' in {self.marker_dict[c]} — invalid PSD shape")
                    continue
            
                try:
                    total = integrate.simpson(P, dx=freq_res)
                    res[c][ch] = {}
                    for b, (f0, f1) in self.bands.items():
                        idx = (self.freqs >= f0) & (self.freqs <= f1)
                        bp = integrate.simpson(P[idx], dx=freq_res)
                        res[c][ch][b] = bp / total
                except Exception as e:
                    print(f"⚠️ Error computing band power for '{ch}' in {self.marker_dict[c]}: {e}")
                    continue

                    
            
        self.relative_power = res
        # flatten to DataFrame
        records = []
        for c in res:
            label = self.marker_dict[c]
            for ch in res[c]:
                row = {'Condition':label,'Channel':ch}
                row.update(res[c][ch])
                records.append(row)
        self.band_df = pd.DataFrame(records)

    def plot(self, condition: str, ylim=(0,150)):
        # find code for condition
        inv = {v:k for k,v in self.marker_dict.items()}
        code = inv.get(condition)
        plt.figure(dpi=120)
        for ch, P in self.psd[code].items():
            if P is None:
                print(f"⚠️ No PSD for channel '{ch}' — skipping")
                continue
            plt.plot(self.freqs, P, label=ch)

        plt.xlim(0, max(self.freqs))
        plt.ylim(*ylim)
        plt.xlabel("Frequency (Hz)")
        plt.ylabel("Power spectral density (µV²/Hz)")
        plt.title(f"{condition} – Welch PSD")
        plt.legend()
        plt.show()

    def plot_all_epochs(self, channel: str = "TP9", ylim=(-100, 100)):
        """
        Plots all EEG trials per stimulus condition for a single channel.
        Each condition gets its own subplot.
        """
        n_cond = len(self.events)
        if n_cond == 0:
            print("⚠️ No conditions available to plot.")
            return
    
        fig, axes = plt.subplots(n_cond, 1, figsize=(12, 4 * n_cond), dpi=100, squeeze=False)
        fig.suptitle(f"EEG epochs – {channel}", fontsize=16)
    
        for i, (code, trials_by_channel) in enumerate(self.events.items()):
            trials = trials_by_channel.get(channel, [])
            ax = axes[i][0]
            for trial in trials:
                t = np.linspace(0, self.length, len(trial))
                ax.plot(t, trial, alpha=0.5)
    
            label = self.marker_dict.get(code, f"Code {code}")
            ax.set_title(f"{label} – {len(trials)} trials")
            ax.set_xlabel("Time (s)")
            ax.set_ylabel("Amplitude (μV)")
            ax.set_ylim(*ylim)
            ax.grid(True)
    
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    def plot_every_trial(self, channel: str = "TP9", ylim=(-100, 100)):
        """
        Creates a separate plot for every individual EEG trial across all stimulus types.
        Each figure shows one response for a given channel.
        """
        total = 0
        for code, channel_trials in self.events.items():
            trials = channel_trials.get(channel, [])
            label = self.marker_dict.get(code, f"Code {code}")
    
            for i, trial in enumerate(trials):
                t = np.linspace(0, self.length, len(trial))
                plt.figure(figsize=(10, 3), dpi=100)
                plt.plot(t, trial, label=f"{label} – Trial {i+1}")
                plt.title(f"{label} – Trial {i+1} ({channel})")
                plt.xlabel("Time (s)")
                plt.ylabel("Amplitude (μV)")
                plt.ylim(*ylim)
                plt.grid(True)
                plt.tight_layout()
                plt.show()
                total += 1
    
        print(f"✅ Plotted {total} trials across all conditions for channel '{channel}'")

    def plot_every_trial_all_channels(self, ylim=(-100, 100)):
        """
        Creates separate plots for every EEG trial across all stimulus types and all channels.
        Each figure shows one response at one channel.
        """
        total = 0
        for code, channel_trials in self.events.items():
            label = self.marker_dict.get(code, f"Code {code}")
    
            for channel, trials in channel_trials.items():
                for i, trial in enumerate(trials):
                    t = np.linspace(0, self.length, len(trial))
                    plt.figure(figsize=(10, 3), dpi=100)
                    plt.plot(t, trial, label=f"{label} – Trial {i+1} – {channel}")
                    plt.title(f"{label} – Trial {i+1} ({channel})")
                    plt.xlabel("Time (s)")
                    plt.ylabel("Amplitude (μV)")
                    plt.ylim(*ylim)
                    plt.grid(True)
                    plt.tight_layout()
                    plt.show()
                    total += 1
    
        print(f"✅ Plotted {total} trial-channel waveforms across all conditions")

    
    #TROUBLESHOOTING
    def diagnose_trials(self):
        """
        Prints a summary of trial counts:
        - Total stimuli per condition from behavioral data
        - Trials injected into EEG windows
        - Trials retained after cleaning
        """
        print("\n📋 Trial Diagnostics:")
    
        # Step 1 — Behavior CSV
        if hasattr(self, 'beh') and 'Stimulus' in self.beh.columns:
            beh_counts = self.beh["Stimulus"].value_counts()
            print("🧪 Behavioral stimulus counts:")
            for stim, count in beh_counts.items():
                print(f"  {stim}: {count}")
        else:
            print("⚠️ Behavioral data not available or missing 'Stimulus' column")
    
        # Step 2 — Injected trials from marker_map
        print("\n🔗 EEG-aligned trials per condition:")
        for code in self.events:
            stim = self.marker_dict.get(code, f"Code {code}")
            trial_count = len(next(iter(self.events[code].values()), []))
            print(f"  {stim}: {trial_count} trials")
    
        # Step 3 — Cleaned windows (after noise filtering)
        print("\n🧹 Clean windows retained per condition:")
        for code in self.windows:
            stim = self.marker_dict.get(code, f"Code {code}")
            ch_counts = {ch: len(ws) for ch, ws in self.windows[code].items()}
            avg_clean = np.mean(list(ch_counts.values()))
            print(f"  {stim}: ~{int(avg_clean)} clean trials (avg per channel)")


    def tag_noise_levels(self, thresholds=(50, 150)):
        """
        Labels each EEG trial's noise level: 'low', 'medium', or 'high',
        based on peak-to-peak amplitude. No trials are removed.
        Results are stored in self.noise_labels[condition][channel] = [labels...]
        """
        self.noise_labels = {}
    
        for code, channel_trials in self.windows.items():
            self.noise_labels[code] = {}
            label = self.marker_dict.get(code, f"Code {code}")
    
            for ch, trials in channel_trials.items():
                level_tags = []
                for trial in trials:
                    ptp = np.ptp(trial)  # Peak-to-peak amplitude
                    if ptp < thresholds[0]:
                        level_tags.append("low")
                    elif ptp < thresholds[1]:
                        level_tags.append("medium")
                    else:
                        level_tags.append("high")
                self.noise_labels[code][ch] = level_tags
    
        print("✅ Noise levels tagged for all trials. No exclusions applied.")


    def get_trials_by_noise(self, levels=("low", "medium"), condition=None, channel="TP9"):
        """
        Retrieves trials matching any of the specified noise levels.
        Args:
            levels (tuple): One or more noise levels ('low', 'medium', 'high')
            condition (str): Optional stimulus label (e.g. 'circle')
            channel (str): EEG channel name (e.g. 'AF8')
        Returns:
            List of EEG trials matching criteria
        """
        if not hasattr(self, "noise_labels"):
            print("⚠️ Noise labels not found. Run `tag_noise_levels()` first.")
            return []
    
        matched_trials = []
    
        for code, ch_labels in self.noise_labels.items():
            stim = self.marker_dict.get(code, f"Code {code}")
            if condition and stim != condition:
                continue
            if channel not in ch_labels:
                continue
    
            labels = ch_labels[channel]
            trials = self.windows[code][channel]
    
            for trial, lbl in zip(trials, labels):
                if lbl in levels:
                    matched_trials.append(trial)
    
        print(f"✅ Retrieved {len(matched_trials)} trials with noise levels {levels}" +
              (f" from '{condition}' @ {channel}" if condition else f" @ {channel}"))
        return matched_trials


    def plot_trials(self, trials, title="EEG Trials", color="slateblue"):
        """
        Plots a list of EEG trials (1D arrays).
        Each trial will be plotted as a separate line.
        """
        import matplotlib.pyplot as plt
    
        if not trials:
            print("⚠️ No trials to plot.")
            return
    
        plt.figure(figsize=(10, 5))
        for i, trial in enumerate(trials):
            plt.plot(trial, alpha=0.5, label=f"Trial {i+1}", color=color)
    
        plt.title(title)
        plt.xlabel("Time (samples)")
        plt.ylabel("Amplitude (μV)")
        plt.grid(True)
        plt.tight_layout()
        plt.show()




    

    def dataframe(self):
        return self.band_df.copy()


In [ ]:
import pandas as pd

# 1. Load EEG (timestamps are already float Unix seconds)
eeg_df = load_eeg(eeg_file)

# 2. Read behavioral log raw
behav_df_raw = pd.read_csv(behav_file)

# ✅ 3. Use timestamps directly — no offset
beh_df = behav_df_raw.copy()
beh_df["marker_ts"] = beh_df["Marker Timestamp"]  # Already in Unix seconds

# 4. Build stimulus label-to-code map
stim_labels = beh_df["Stimulus"].unique()
marker_map = {label: i + 1 for i, label in enumerate(stim_labels)}

# 5. Run pipeline
session = Recording(
    eeg_df,
    beh_df,
    marker_map,
    length=20,
    min_freq=1,
    max_freq=30
)


In [ ]:
session.diagnose_trials()


In [ ]:
session.dataframe()

In [ ]:
for condition in session.dataframe()["Condition"].unique():
    session.plot(condition, ylim=(0, 50))

